<a href="https://colab.research.google.com/github/Kaan-wq/ml_tokamak/blob/main/NN_general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
import os
import glob
import numpy as np
from typing import Iterable
import pickle
import tensorflow as tf
from scipy.fft import fft, ifft
from sklearn.decomposition import PCA

import keras
import keras_tuner as kt
from kerastuner import RandomSearch
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras import regularizers

#**I - Preprocessing of the data**

Below is the preprocessing pipeline of the data. \\
Essentially, we feature engineer a few columns, normalize the data and finally, we split it into a training and testing set.

In [14]:
# load dataset using pickle
import pickle
with open("../data/dataset_disruption_characterization.pickle", "rb") as f:
    dataset = pickle.load(f)

Here we load the data from the drive and put it into a more practical **data structure**. \\
We add a column <code>['IPE']</code> which represents the current difference between the reference and actual currents.

In [15]:
df_data = pd.DataFrame()

# Loop through each entry in the dataset
for i, entry in enumerate(dataset):
    # Extract data and label from the current entry
    d = entry['x']
    label = entry['y']
    metadata = entry['metadata']
    event = metadata['time_event']

    # Create a DataFrame for the current entry
    df = pd.DataFrame(d['data'], columns=d['columns'])

    # Add the IPE column
    df['IPE'] = np.abs(df['IPLA'] - df['IPref'])

    # Define columns to perform feeture engineering on
    column = ['IPLA', 'IPref', 'IPE', 'ECEcore', 'SSXcore', 'LI', 'Q95', 'ZMAG', 'Vloop']

    # Add the Fourier Transform columns
    for col in column:
      # Perform the Fourier Transform
      df[col + '_fft'] = np.fft.fft(df[col])

      # The result is complex numbers, you might want to take absolute value (magnitude) of the result
      df[col + '_fft_abs'] = np.abs(df[col + '_fft'])

      # Drop the column with complex numbers
      df = df.drop(columns=[col + '_fft'])


    # Add the first order derivative columns
    for col in column:
      # Perform the first order derivative
      df[col + '_der'] = np.gradient(df[col])

    # Add the second order derivative columns
    for col in column:
      # Perform the second order derivative
      df[col + '_der2'] = np.gradient(df[col + '_der'])
    
    # Add the time column
    df['Time'] = d['time']

    # Add the Frame, Event, Label, Shot and Window columns
    df['Frame'] = range(0, 20)
    df['Event'] = event if event else 0

    if event:
      #Find closest points to time_event
      differences = np.abs(df['Time'] - event)
      closest_indices = np.argsort(differences)[:20]

      #Assign labels to closest points
      df['Label'] = 0
      df.loc[closest_indices, 'Label'] = label
    else:
      df['Label'] = label

    df['Shot'] = metadata['shot']
    df['Window'] = i  # Add the window number

    contains_nan = df.isna().any().any()

    if not contains_nan:
      # Append the current DataFrame to the main DataFrame
      df_data = pd.concat([df_data, df], ignore_index=True)


df_data = df_data.dropna()
display(df_data)
print("Labels count:")
print(f"{df_data['Label'].value_counts()}")

base_col = df_data.columns

,IPLA,IPref,ECEcore,SSXcore,LI,Q95,ZMAG,Vloop,IPE,IPLA_fft_abs,...,LI_der2,Q95_der2,ZMAG_der2,Vloop_der2,Time,Frame,Event,Label,Shot,Window
0,1989456.750,1999500.0,1740.929077,0.008850,1.191489,3.874169,0.30388,-0.519496,10043.250,3.978742e+07,...,0.0,0.0,0.0,-0.019993,10.361,0,0.0,0,81206,0
1,1989606.250,1999500.0,1744.737427,0.008850,1.191489,3.874169,0.30388,-0.758418,9893.750,2.153723e+03,...,0.0,0.0,0.0,0.119461,10.362,1,0.0,0,81206,0
2,1988484.000,1999500.0,1756.823730,0.008698,1.191489,3.874169,0.30388,-1.037327,11016.000,4.264020e+03,...,0.0,0.0,0.0,0.448103,10.363,2,0.0,0,81206,0
3,1989329.625,1999500.0,1756.823730,0.008469,1.191489,3.874169,0.30388,-0.758418,10170.375,5.440219e+03,...,0.0,0.0,0.0,-0.159448,10.364,3,0.0,0,81206,0
4,1990532.250,1999500.0,1746.057251,0.008850,1.191489,3.874169,0.30388,0.237256,8967.750,4.025099e+03,...,0.0,0.0,0.0,-0.537741,10.365,4,0.0,0,81206,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17975,3993662.500,4000500.0,6282.263672,1.416517,0.808256,2.898196,0.32961,-0.316564,6837.500,1.188296e+04,...,0.0,0.0,0.0,0.138705,8.622,15,0.0,0,98005,905
17976,3995614.250,4000500.0,6303.055664,1.410871,0.808256,2.898196,0.32961,-0.237256,4885.750,6.513881e+03,...,0.0,0.0,0.0,0.198435,8.623,16,0.0,0,98005,905
17977,3994348.000,4000500.0,6277.216309,1.402783,0.808256,2.898196,0.32961,-0.238922,6152.000,9.370845e+03,...,0.0,0.0,0.0,0.000416,8.624,17,0.0,0,98005,905
17978,3997734.500,4000500.0,6286.124512,1.391034,0.808256,2.898196,0.32961,0.279242,2765.500,1.626988e+04,...,0.0,0.0,0.0,-0.348553,8.625,18,0.0,0,98005,905


Labels count:
Label
0    12840
1     4140
2     1000
Name: count, dtype: int64


In [16]:
shot = df_data[df_data['Label'] == 2]['Shot'].unique()[6]

# Filter the DataFrame to only include a specific shot
df_shot = df_data[df_data['Shot'] == shot]

# Filter the DataFrame to only include rows with labels clustered
df_label_2 = df_shot[df_shot['Label'] == 2]
df_label_1 = df_shot[df_shot['Label'] == 1]
df_label_0 = df_shot[df_shot['Label'] == 0]

# Select a specific window for each label
window_number_2 = df_label_2['Window'].unique()[0] 
df_window_2 = df_label_2[df_label_2['Window'] == window_number_2]

window_number_1 = df_label_1['Window'].unique()[0] 
df_window_1 = df_label_1[df_label_1['Window'] == window_number_1]

window_number_0 = df_label_0['Window'].unique()[0]
df_window_0 = df_label_0[df_label_0['Window'] == window_number_0]

# Plot each feature across time
features = df_data.columns.drop(['Label', 'Window', 'Shot', 'Frame', 'Time', 'Event'])  # Exclude non-feature columns

# Print the time of the event
print("Time of the event for each label is :")
print(f"Label 2: {df_window_2['Event'].unique()[0]}")
print(f"Label 1: {df_window_1['Event'].unique()[0]}")
print(f"Label 0: {df_window_0['Event'].unique()[0]}")

# Create a subplot for each feature
fig, axs = plt.subplots(len(features), 3, figsize=(20, 6*len(features)))

for i, feature in enumerate(features):
    axs[i, 0].plot(df_window_2['Time'], df_window_2[feature], label=f'Label 2')
    axs[i, 0].set_title(f'Feature: {feature} over Time for Window: {window_number_2}')
    axs[i, 0].set_xlabel('Time')
    axs[i, 0].set_ylabel(feature)
    axs[i, 0].legend()

    axs[i, 1].plot(df_window_1['Time'], df_window_1[feature], label=f'Label 1')
    axs[i, 1].set_title(f'Feature: {feature} over Time for Window: {window_number_1}')
    axs[i, 1].set_xlabel('Time')
    axs[i, 1].set_ylabel(feature)
    axs[i, 1].legend()

    axs[i, 2].plot(df_window_0['Time'], df_window_0[feature], label=f'Label 0')
    axs[i, 2].set_title(f'Feature: {feature} over Time for Window: {window_number_0}')
    axs[i, 2].set_xlabel('Time')
    axs[i, 2].set_ylabel(feature)
    axs[i, 2].legend()

plt.tight_layout()
plt.show()

# Save the plot in an image file
fig.savefig('../data/feature_time.png')

In [17]:
df_disr = df_data[df_data['Label'] != 0].drop(columns=['Window', 'Shot', 'Frame', 'Time', 'Event'])
df_disr.corr()['Label'].round(3).sort_values(ascending=False)

Label              1.000
Vloop_fft_abs      0.559
IPLA               0.309
IPLA_der2          0.212
Vloop_der2         0.203
Vloop              0.187
IPref              0.185
IPLA_der           0.181
Vloop_der          0.173
IPE_der2           0.065
IPE_der            0.054
IPE                0.037
IPLA_fft_abs       0.036
IPref_fft_abs      0.022
IPE_fft_abs        0.014
ECEcore_fft_abs    0.013
IPref_der          0.012
LI_der2            0.008
LI_der             0.008
ZMAG_der           0.001
ZMAG_der2         -0.001
IPref_der2        -0.002
SSXcore_fft_abs   -0.004
LI_fft_abs        -0.005
Q95_der2          -0.005
ZMAG_fft_abs      -0.013
LI                -0.018
Q95_fft_abs       -0.022
Q95_der           -0.025
ZMAG              -0.057
SSXcore_der2      -0.071
SSXcore_der       -0.081
SSXcore           -0.086
ECEcore           -0.093
ECEcore_der2      -0.108
ECEcore_der       -0.146
Q95               -0.268
Name: Label, dtype: float64

In [18]:
df_data.columns

Index(['IPLA', 'IPref', 'ECEcore', 'SSXcore', 'LI', 'Q95', 'ZMAG', 'Vloop',
       'IPE', 'IPLA_fft_abs', 'IPref_fft_abs', 'IPE_fft_abs',
       'ECEcore_fft_abs', 'SSXcore_fft_abs', 'LI_fft_abs', 'Q95_fft_abs',
       'ZMAG_fft_abs', 'Vloop_fft_abs', 'IPLA_der', 'IPref_der', 'IPE_der',
       'ECEcore_der', 'SSXcore_der', 'LI_der', 'Q95_der', 'ZMAG_der',
       'Vloop_der', 'IPLA_der2', 'IPref_der2', 'IPE_der2', 'ECEcore_der2',
       'SSXcore_der2', 'LI_der2', 'Q95_der2', 'ZMAG_der2', 'Vloop_der2',
       'Time', 'Frame', 'Event', 'Label', 'Shot', 'Window'],
      dtype='object')

We **normalize and split** into training and test sets before feeding it into our **Neural Network**.

In [21]:
# Split 'Shot' values into training and test sets and create DataFrames
shot_train, shot_test = train_test_split(df_data['Shot'].unique(), test_size=0.2, random_state=42)
train_df = df_data[df_data['Shot'].isin(shot_train)]
test_df = df_data[df_data['Shot'].isin(shot_test)]

# Group by 'Window' and 'Shot', and reshape the data
train_df_grouped = train_df.sort_values(['Shot', 'Window', 'Time']).groupby(['Shot', 'Window'])
test_df_grouped = test_df.sort_values(['Shot', 'Window', 'Time']).groupby(['Shot', 'Window'])

# Prepare lists to hold sequences
X_train_grouped, y_train_grouped = [], []
X_test_grouped, y_test_grouped = [], []

# Generate sequences for training data
for _, group in train_df_grouped:
    X_train_grouped.append(group.drop(columns=['Frame', 'Event', 'Label', 'Shot', 'Window']).values)
    y_train_grouped.append(group['Label'].values[0])  # Assuming all labels in a group are the same

# Generate sequences for testing data
for _, group in test_df_grouped:
    X_test_grouped.append(group.drop(columns=['Frame', 'Event', 'Label', 'Shot', 'Window']).values)
    y_test_grouped.append(group['Label'].values[0])  # Assuming all labels in a group are the same

# Convert lists to numpy arrays
X_train_grouped = np.array(X_train_grouped)
y_train_grouped = np.array(y_train_grouped)
X_test_grouped = np.array(X_test_grouped)
y_test_grouped = np.array(y_test_grouped)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_grouped.reshape(-1, X_train_grouped.shape[-1])).reshape(X_train_grouped.shape)
X_test = scaler.transform(X_test_grouped.reshape(-1, X_test_grouped.shape[-1])).reshape(X_test_grouped.shape)

# Convert labels to categorical
y_train = to_categorical(y_train_grouped)
y_test = to_categorical(y_test_grouped)

In [22]:
X_train.shape

(737, 20, 37)

#**II - Model tuning**

Here we use the hyperparamter tuner from keras to find the best model architecture.

In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam

# Function for creating a residual block
def residual_block(x, filters, kernel_size):
    # Save the input value. Apply a 1x1 convolution to match the number of channels.
    residual = Conv1D(filters, 1, padding='same')(x)

    # Convolution, batch normalization, and ReLU activation (repeated three times)
    for _ in range(3):
        x = Conv1D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # Add the residual (input) to the output
    x = Add()([x, residual])

    return x

# Input shape: (timesteps, features)
input_shape = (20, 37)

# Input layer
input_layer = Input(shape=input_shape)

# Create three residual blocks with varying kernel sizes
x = residual_block(input_layer, 64, 8)
x = residual_block(x, 64, 5)
x = residual_block(x, 64, 3)

# Global Average Pooling layer
x = GlobalAveragePooling1D()(x)

# Final softmax classifier
output_layer = Dense(3, activation='softmax')(x)

# Create the model and compile it
model = Model(inputs=input_layer, outputs=output_layer)

# Create the optimizer
optimizer = Adam(learning_rate=0.0000001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', tfa.metrics.F1Score(num_classes=3, average=None)])

# Print the model summary
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 20, 37)]             0         []                            
                                                                                                  
 conv1d_25 (Conv1D)          (None, 20, 64)               19008     ['input_3[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 20, 64)               256       ['conv1d_25[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 20, 64)               0         ['batch_normalization_18

In [26]:
# Train the model
history = model.fit(X_train, y_train, epochs=10000, validation_data=(X_test, y_test))

# Plot the losses
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Print the F1-score on the validation set
_, _, f1_score = model.evaluate(X_test, y_test)
print('F1-score on validation set:', f1_score)

Epoch 1/10000
24/24 [==============================] - 5s 33ms/step - loss: 0.9118 - accuracy: 0.6187 - f1_score: 0.3743 - val_loss: 0.9313 - val_accuracy: 0.6605 - val_f1_score: 0.3756
Epoch 2/10000
24/24 [==============================] - 0s 16ms/step - loss: 0.9097 - accuracy: 0.6255 - f1_score: 0.3915 - val_loss: 0.9388 - val_accuracy: 0.6667 - val_f1_score: 0.3791
Epoch 3/10000
24/24 [==============================] - 0s 16ms/step - loss: 0.9213 - accuracy: 0.6174 - f1_score: 0.3831 - val_loss: 0.9298 - val_accuracy: 0.6667 - val_f1_score: 0.3498
Epoch 4/10000
24/24 [==============================] - 0s 16ms/step - loss: 0.9183 - accuracy: 0.6309 - f1_score: 0.4097 - val_loss: 0.9200 - val_accuracy: 0.7037 - val_f1_score: 0.3848
Epoch 5/10000
24/24 [==============================] - 0s 16ms/step - loss: 0.8961 - accuracy: 0.6445 - f1_score: 0.3889 - val_loss: 0.9147 - val_accuracy: 0.7284 - val_f1_score: 0.3979
Epoch 6/10000
24/24 [==============================] - 0s 16ms/step - 